# Model Development for Molecular Solubility Prediction

This notebook demonstrates the model development process for predicting molecular solubility.

We'll perform the following tasks:
1. Load the processed data with molecular descriptors
2. Split the data into training and testing sets
3. Train multiple models and optimize hyperparameters
4. Evaluate model performance
5. Analyze feature importance
6. Make predictions on new compounds

In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Draw

# Import scikit-learn modules
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor

# Import from our own modules
import sys
sys.path.append('..')
from src.models.train_model import train_test_split_data, train_random_forest, evaluate_model
from src.visualization.visualize import plot_feature_importance, plot_actual_vs_predicted, plot_distributions

# Set up plotting
%matplotlib inline
plt.style.use('seaborn-whitegrid')
sns.set_palette('viridis')

## 1. Load Processed Data

Let's load the processed data we created in the previous notebook.

In [ ]:
# Load the processed data
features_df = pd.read_csv('../data/processed/molecular_features.csv')

# Display the first few rows
features_df.head()

In [ ]:
# Check the shape of the features dataframe
print(f"Feature dataframe shape: {features_df.shape}")

# Check for missing values
missing_values = features_df.isnull().sum()
print("
Features with missing values:")
print(missing_values[missing_values > 0] if any(missing_values > 0) else "None")

## 2. Prepare Data for Modeling

Now we'll split the data into features (X) and target (y), and then into training and testing sets.

In [ ]:
# Separate features and target
X = features_df.drop(['SMILES', 'logS_exp'], axis=1)
y = features_df['logS_exp']

# Apply feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split_data(X_scaled, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

## 3. Baseline Models

Let's train and evaluate several baseline models to establish a performance benchmark.

In [ ]:
# Define models to evaluate
models = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# Train and evaluate each model using cross-validation
cv_results = {}

for name, model in models.items():
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    cv_results[name] = {
        'mean_rmse': rmse_scores.mean(),
        'std_rmse': rmse_scores.std()
    }
    print(f"{name}: Mean RMSE = {rmse_scores.mean():.4f} (±{rmse_scores.std():.4f})")

In [ ]:
# Visualize cross-validation results
cv_df = pd.DataFrame({
    'Model': list(cv_results.keys()),
    'Mean RMSE': [result['mean_rmse'] for result in cv_results.values()],
    'Std RMSE': [result['std_rmse'] for result in cv_results.values()]
})

plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='Mean RMSE', data=cv_df)
plt.errorbar(
    x=np.arange(len(cv_df)),
    y=cv_df['Mean RMSE'],
    yerr=cv_df['Std RMSE'],
    fmt='none',
    color='black',
    capsize=5
)
plt.title('Cross-Validation RMSE for Different Models')
plt.ylabel('RMSE')
plt.grid(axis='y', alpha=0.3)
plt.show()

## 4. Hyperparameter Tuning

Based on the baseline results, let's optimize the hyperparameters of the best-performing model.

In [ ]:
# Define parameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Train Random Forest with hyperparameter tuning
rf_model = train_random_forest(X_train, y_train, param_grid=rf_param_grid)

## 5. Model Evaluation

Now let's evaluate the optimized model on the test set.

In [ ]:
# Evaluate the optimized model
metrics = evaluate_model(rf_model, X_test, y_test)

In [ ]:
# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Plot actual vs predicted values
fig = plot_actual_vs_predicted(y_test, y_pred)
plt.show()

In [ ]:
# Plot distributions of actual and predicted values
fig = plot_distributions(y_test, y_pred)
plt.show()

## 6. Feature Importance Analysis

Let's analyze which molecular descriptors are most important for solubility prediction.

In [ ]:
# Plot feature importance
fig = plot_feature_importance(rf_model, X.columns, top_n=15)
plt.show()

## 7. Prediction on New Compounds

Finally, let's demonstrate how to use the trained model to predict solubility for new compounds.

In [ ]:
# Import prediction module
from src.models.predict_model import predict_solubility

# Example SMILES for new compounds
new_smiles = [
    'CC(=O)OC1=CC=CC=C1C(=O)O',  # Aspirin
    'CN1C=NC2=C1C(=O)N(C(=O)N2C)C',  # Caffeine
    'CCC1(CC)C(=O)NC(=O)N(C)C1=O',  # Phenobarbital
    'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O',  # Ibuprofen
    'CN1C2CCC1CC(C2)OC(=O)C(CO)C3=CC=CC=C3'  # Cocaine
]

# Make predictions
predictions = predict_solubility(rf_model, new_smiles, scaler=scaler)
predictions

In [ ]:
# Visualize the predicted compounds
mols = [Chem.MolFromSmiles(smiles) for smiles in predictions['SMILES']]
legends = [f"{Chem.MolToSmiles(mol, isomericSmiles=False)}
LogS: {pred:.2f}" 
           for mol, pred in zip(mols, predictions['Predicted_Solubility'])]

img = Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(300, 300), legends=legends)
display(img)

## 8. Save Model and Scaler

Let's save the trained model and scaler for future use.

In [ ]:
# Import function to save model
from src.models.train_model import save_model

# Create directories if they don't exist
os.makedirs('../models', exist_ok=True)

# Save model
save_model(rf_model, '../models/random_forest_model.pkl')

# Save scaler
save_model(scaler, '../models/scaler.pkl')

## Summary

In this notebook, we have:

1. Loaded the processed molecular descriptor data
2. Prepared the data for modeling by scaling features and splitting into training/testing sets
3. Trained and evaluated several baseline models
4. Optimized the hyperparameters of the Random Forest model
5. Evaluated the optimized model on the test set
6. Analyzed feature importance to understand which descriptors are most predictive
7. Demonstrated how to make predictions on new compounds
8. Saved the trained model and scaler for future use

The final model achieved good performance in predicting molecular solubility based on chemical descriptors calculated from molecular structures. This demonstrates a complete machine learning workflow for structure-property relationship modeling.